# Sistemas de control con Python

In [1]:
from scipy.signal import lti
#scipy.signal permite trabajar sistemas lineales en tiempo continuo (LTI)
import matplotlib.pyplot as plt
# matplotlib.pyplot permite realizar gráficas
import sympy as sy
# sympy permite trabajar con matemáticas simbólicas
from sympy import Symbol
import numpy as np
from ipywidgets import interact, fixed

In [2]:
def pretysys(sys):
    s = sy.symbols('s')
    num = sy.Poly(sys.num, s)
    den = sy.Poly(sys.den, s)
    return num/den

In [3]:
def serie(sys1, sys2):
    num = np.polymul(sys1.num, sys2.num)
    den = np.polymul(sys1.den, sys2.den)
    return lti(num,den)

In [4]:
a = lti([2],[1,2])
b = lti([1], [1,2,3])

In [5]:
pretysys(a)

2.0/(1.0*s + 2.0)

In [6]:
pretysys(b)

1.0/(1.0*s**2 + 2.0*s + 3.0)

In [7]:
pretysys(serie(a,b))

2.0/(1.0*s**3 + 4.0*s**2 + 7.0*s + 6.0)

funcion_t_open_loop = $G(s)$

fun_t_feedback = $H(s)$

$$GH(s) = \frac{G(s)}{1+G(s)\cdot H(s)} $$    

In [8]:
def feedback(sys1, sys2):
    num=np.polymul(sys1.num, sys2.den)
    a = np.polymul(sys1.num, sys2.num)
    b = np.polymul(sys1.den, sys2.den)
    den = np.polyadd(a,b)
    return lti(num,den)

In [9]:
pretysys(feedback(a,b))

(2.0*s**2 + 4.0*s + 6.0)/(1.0*s**3 + 4.0*s**2 + 7.0*s + 8.0)

# PID interactivo

In [10]:
def param(t,y):
  n=0 #indice del tiempo de levantamiento
  vf=y[-1]#valor final de la respuesta
  tp=t[np.argmax(y)] # tiempo pico
  Ma=(max(y)-y[-1])/y[-1]#porcentaje de maximo sobreimpulso

  while y[n]<vf:
    n+=1
  
  tl=t[n]#tiempo de levantamiento

  e=abs(y-vf) #error absoluto

  ei=e[::-1]#error en orden invertido
  
  m=1 #indice del tiempo de establecimiento

  while ei[m]<2*vf/100:
    m+=1
  
  ts=t[-m]# tiempo de establecimiento al 2%

  return print(f'%Amax={Ma*100}\nVf={vf}\ntp={tp}\ntl={tl}\nts={ts}')

In [11]:
def pid_tuner(sys,ret=0.0,kp=10,ki=10,kd=10,delta_P=0.01,delta_I=0.01,delta_D=0.01):

  @interact(kp=(0.001,kp,delta_P),ki=(0.001,ki,delta_I),kd=(0.001,kd,delta_D),param_orig=['Yes','No'],param_cont=['Yes','No'])
  def CONTROLAR(kp,ki,kd,param_orig,param_cont):
    ts,ys=sys.step()
    plt.plot(ts,ys,label='Sistema sin controlar')
    plt.legend(loc='best')
    cont=lti([kd,kp,ki],[1,0])
    ser=serie(cont,sys)
    fed=feedback(ser,lti([1],[1]))
    tc,yc=fed.step(T=ts)
    plt.plot(tc,yc,label='Sistema controlado')
    plt.legend(loc='best')
    plt.title('PID')
    plt.xlabel('T(s)')
    plt.ylabel('Amplitud')
    plt.grid()
    plt.show
    if param_orig=="Yes":
      print('\n Parametros del sistema origianal')
      param(ts,ys)
    
    if param_cont=='Yes':
      print('\n Parametros del sistema controlado')
      param(tc,yc)

In [12]:
sys=lti([2],[1,2,3])

In [13]:
pid_tuner(sys)

interactive(children=(FloatSlider(value=4.9910000000000005, description='kp', max=10.0, min=0.001, step=0.01),…